<a href="https://colab.research.google.com/github/Sree051203/Sree051203/blob/main/AI_Supply_Bot_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import random
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from google.colab import files

In [11]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print("Dataset loaded successfully!")
print(df.head())

Saving Supply Chain.zip to Supply Chain (3).zip
Dataset loaded successfully!
  Product type   SKU      Price  Availability  Number of products sold  \
0     haircare  SKU0  69.808006            55                      802   
1     skincare  SKU1  14.843523            95                      736   
2     haircare  SKU2  11.319683            34                        8   
3     skincare  SKU3  61.163343            68                       83   
4     skincare  SKU4   4.805496            26                      871   

   Revenue generated Customer demographics  Stock levels  Lead times  \
0        8661.996792            Non-binary            58           7   
1        7460.900065                Female            53          30   
2        9577.749626               Unknown             1          10   
3        7766.836426            Non-binary            23          13   
4        2686.505152            Non-binary             5           3   

   Order quantities  ...  Location Lead time 

In [12]:
df = df[['Product type', 'Price', 'Stock levels', 'Lead times', 'Order quantities']]

df = df.dropna()
df = pd.get_dummies(df, columns=['Product type'])

print("Preprocessed Dataset:")
print(df.head())

Preprocessed Dataset:
       Price  Stock levels  Lead times  Order quantities  \
0  69.808006            58           7                96   
1  14.843523            53          30                37   
2  11.319683             1          10                88   
3  61.163343            23          13                59   
4   4.805496             5           3                56   

   Product type_cosmetics  Product type_haircare  Product type_skincare  
0                   False                   True                  False  
1                   False                  False                   True  
2                   False                   True                  False  
3                   False                  False                   True  
4                   False                  False                   True  


In [13]:
target = "Order quantities"

X = df.drop(target, axis=1)
y = df[target]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = LinearRegression()
model.fit(X_train, y_train)

print("Model trained successfully!")

Model trained successfully!


In [14]:
# Prediction
y_pred = model.predict(X_test)

# Evaluatuation
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 790.8026799167562


In [15]:
def simulate_supplier_disruption(probability=0.1):
    """Simulates supplier disruption with a given probability."""
    return np.random.rand() < probability

reorder_threshold = 40

num_suppliers = 3
suppliers = [f"Supplier_{i}" for i in range(num_suppliers)]

product_suppliers = {}

product_types = [col.split('_')[1] for col in df.columns if 'Product type' in col]

print("The Product Types " + str(product_types))

for product_type in product_types:
  product_suppliers[product_type] = random.choice(suppliers)

print("Initial Product Suppliers:", product_suppliers)

disrupted_products = set()

total_orders = 0
disruption_count = 0
supplier_switches = 0

#Simulating demand and checking stock levels
for index, row in df.iterrows():
    product_type = None
    for p_type in product_types:
      if row[f'Product type_{p_type}']:
        product_type = p_type
        break

    stock_levels = row['Stock levels']
    order_quantities = row['Order quantities']

    if stock_levels < reorder_threshold:
        total_orders += 1

        disrupted = simulate_supplier_disruption()

        if disrupted and product_type not in disrupted_products:
            print(f" Supplier Disrupted for {product_type}! Looking for alternatives...")
            disruption_count += 1
            disrupted_products.add(product_type)
            available_suppliers = [s for s in suppliers if s != product_suppliers[product_type]]
            if available_suppliers:
                new_supplier = random.choice(available_suppliers)
                print(f"  Switching to {new_supplier} for {product_type}")
                product_suppliers[product_type] = new_supplier
                supplier_switches += 1
            else:
                print("  No alternative suppliers available!")

        else:
            print(f"Order placed for {product_type} with {product_suppliers[product_type]} to restock.")
        df.at[index, 'Stock levels'] = df.at[index, 'Stock levels'] + order_quantities
print("\n--- Simulation Report ---")
print(f"Total Orders Placed: {total_orders}")
print(f"Total Disruptions: {disruption_count}")
print(f"Total Supplier Switches: {supplier_switches}")
print("Updated Product Suppliers:", product_suppliers)
print("Inventory Management Simulation Completed!")
print("Disrupted Products:", disrupted_products)

The Product Types ['cosmetics', 'haircare', 'skincare']
Initial Product Suppliers: {'cosmetics': 'Supplier_2', 'haircare': 'Supplier_1', 'skincare': 'Supplier_0'}
Order placed for haircare with Supplier_1 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for skincare with Supplier_0 to restock.
 Supplier Disrupted for cosmetics! Looking for alternatives...
  Switching to Supplier_0 for cosmetics
Order placed for skincare with Supplier_0 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for haircare with Supplier_1 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for haircare with Supplier_1 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for cosmetics with Supplier_0 to restock.
Order placed for skincare with Supplier_0 to restock.
Order placed for skincare with 